In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import evosax

import json


In [24]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.opt

import xylo.tuning
import xylo.database

In [27]:
tuning = xylo.tuning.yamaha_YX500R
# wood = t.Wood.make_E_nu(E = 29.100e9, nu = 8.20, rho = 1059)
# init_mean = None

In [28]:
options = xylo.opt.Options(num_generations = 100, strategy = evosax.OpenES(popsize = 2000, num_dims = 3), absolute_tolerance = 1e-3)

In [49]:
for num_dims in range(7,10):
  options = xylo.opt.Options(num_generations = 100, strategy = evosax.OpenES(popsize = 2000, num_dims = num_dims), absolute_tolerance = 1e-2)
  init_mean = None

  # for note in range(57, 89):
  for note in range(70, 89):
    db = xylo.database.get()
    match db.get_best_for_dims(note, num_dims):
      case None:
        pass
      case e:
        init_mean = e

    options_ = options._replace(init_mean = init_mean)
    # options_ = options

    bar = db.get_bar(note)
    wood = db.get_wood(note)

    fundamental = tuning.note_to_freq(note)
    partials, weights = tuning.note_to_weights(note)
    print(f"weights {weights}")
    sol = xylo.opt.optimize_geometry(bar, wood, options_, fundamental, jnp.array(partials), jnp.array(weights))
    spline = sol.best_member

    xylo.database.with_db(lambda db: db.set_best_for_dims(note, num_dims, sol.best_member.tolist(), sol.best_fitness.tolist()))

    init_mean = sol.best_member

    sections = xylo.cut.spline(bar, spline)
    swp = xylo.sweep.sweep(wood, bar, sections, t.sweep_default)

    print(note)
    print("---------------")
    print(swp.harmonics / fundamental)
    print("")


weights [1.0, 0.3, 0.1]
iteration 0
[1.         0.53185845 0.9759024  0.99884275 0.83953474 0.62086857
 0.18003138] 0.034332894
[1486.94371226 4402.249834   6218.49651555] [1.00016066 2.96107852 4.18273773] 0.23613499999999998
iteration 10
[1.         0.60594492 1.         1.         0.84281532 0.57497054
 0.19098376] 0.03370078
[1486.63238976 4412.21885085 6202.65812735] [0.99995125 2.96778397 4.17208438] 0.23613499999999998
iteration 20
[1.         0.69339539 1.         1.         0.82856736 0.61046624
 0.23765315] 0.03309795
[1486.33387888 4414.335927   6248.96161197] [0.99975047 2.96920797 4.20322942] 0.23613499999999998
iteration 30
[1.         0.78918888 1.         1.         0.80685726 0.61991737
 0.27240861] 0.032578588
[1486.59779837 4423.74217677 6235.27611943] [0.99992799 2.97553488 4.19402417] 0.23613499999999998
iteration 40
[1.         0.7409651  1.         1.         0.74486213 0.6101978
 0.22585949] 0.032087974
[1486.45430816 4438.14824835 6195.52327966] [0.99983147 2.9